In [1]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [2]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [3]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.RNN as rnn

class GRU(rnn.RNN):

    def __init__(self, D, H, char2idx, idx2char):
        super().__init__(D, H, char2idx, idx2char)

    def forward(self, X, h_old, train=True):
        m = self.model
        Wz, Wh, Wy = m['Wz'], m['Wh'], m['Wy']
        bz, bh, by = m['bz'], m['bh'], m['by']

        X_one_hot = np.zeros(self.D)
        X_one_hot[X] = 1.
        X_one_hot = X_one_hot.reshape(1, -1)

        # concat: [h, x]
        X = np.column_stack((h_old, X_one_hot))

        # gate: h_prob
        hz, hz_cache = l.fc_forward(X, Wz, bz)
        hz, hz_sigm_cache = l.sigmoid_forward(hz)

        # signal: h_pred
        hh, hh_cache = l.fc_forward(X, Wh, bh)
        hh, hh_tanh_cache = l.tanh_forward(hh)

        # h_next and y_pred
        h = h_old + hz * (hh - h_old)
        y, y_cache = l.fc_forward(h, Wy, by)

        cache = h_old, X, hz, hz_cache, hz_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache

        if not train:
            y = util.softmax(y)

        return y, h, cache

    def backward(self, y_pred, y_train, dh_next, cache):
        h_old, X, hz, hz_cache, hz_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache = cache

        dy = loss_fun.dcross_entropy(y_pred, y_train)

        # h_next and y_pred
        dh, dWy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dh_old1 = (1. - hz) * dh

        # signal: h_pred
        dhh = hz * dh
        dhh = l.tanh_backward(dhh, hh_tanh_cache)
        dX, dWh, dbh = l.fc_backward(dhh, hh_cache)
        dh_old2 = dX[:, :self.H]

        # gate: h_prob
        dhz = (hh - h_old) * dh
        dhz = l.sigmoid_backward(dhz, hz_sigm_cache)
        dX, dWz, dbz = l.fc_backward(dhz, hz_cache)
        dh_old3 = dX[:, :self.H]

        # concat: [h, x]
        dh_next = dh_old1 + dh_old2 + dh_old3

        grad = dict(Wz=dWz, Wh=dWh, Wy=dWy, bz=dbz, bh=dbh, by=dby)

        return grad, dh_next

    def _init_model(self, D, C, H):
        Z = H + D

        self.model = dict(
            Wz=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wh=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wy=np.random.randn(H, D) / np.sqrt(D / 2.),
            bz=np.zeros((1, H)),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
        )

In [6]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 10
n_iter = 1000000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [7]:
import impl.LSTM as lstm
import impl.GRU as gru
import impl.RNN as rnn

# net = nn.LSTM(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.RNN(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
net = GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.GRU(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [8]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step,
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.3942
 anexd s, lhe  sre oPy g wtmb rx, cs ytepesifeshein ian Te -iehe 2astind nehe nxpan' nd the oango- H


Iter-2000 loss: 2.7409
ont rnd les' 2oTut ang s, the iyind opilawse wf an erlegeso uleny 1n why  slyy Ririsn wortnin carlee


Iter-3000 loss: 2.3087
r, if Japan's firom-sl -reiJanas latheccin Ooem checichue in "ank in keo
9y, Sirts and Japanese weud


Iter-4000 loss: 2.0078
uthy anferoJana is a. Japan coutro furke 1ion. Japan wageDS ares, on 12t5 canch-fyunttyt tyant Japan


Iter-5000 loss: 1.7784
c, porlat cilion , colamat, fh Fndmburcowad Noppon in the the Worf h7 lamI biuc. If ofefIOryd avese 


Iter-6000 loss: 1.6020
omlland iw the 10th a flitures wed Sunton keowed ch popero'n The porldCrvpHomeofAlt. bf whicenand of


Iter-7000 loss: 1.4321
a e UMestu-Japanese cald of eroxitn ofethe Goun the PEmremare Sthe tirn est poper, ghes. recona wan 


Iter-8000 loss: 1.3043
indErSed ind ronAstancyatithc Emperola. Japan es orer2059Teky wostliwan", s0lloaming in 1947 vi

Iter-25000 loss: 0.5834
riest Chinalecoll Itee bo perigest uro echbe G8, whited suxthe wainttith onturer. Afital pien. Japan


Iter-26000 loss: 0.5586
 surrokeons in the Sea er Tokyo wotstorud cents warges, was recyd wand Shacand canitine in the sour.


Iter-27000 loss: 0.5419
hoptombe tol orean, wosh shanekean "reater and the loped, the tornd the aing saintaingts repflita is


Iter-28000 loss: 0.5317
vich the ed, the Russo-Japana worllith has thoth the Sea of in the Fiuth laggess ciss of ons ond int


Iter-29000 loss: 0.5140
ighest mbomed chec mainoly Cwhothe fiont. hissoled peccokakiowand fixut in 1941, which chargesturing


Iter-30000 loss: 0.5075
f World Worth largest of. The 12st Coran, whict riho, was revilobilacgbeatery in East Asint rrh the 


Iter-31000 loss: 0.4738
aines the G7ov porudhiclawas ped of inch85kal Chinase world's of Japan's lemaits can isel perald of 


Iter-32000 loss: 0.4648
kkailo which cher Indexcan chend its rirstarug the world's Rivromid vopndud ante tor le

Iter-49000 loss: 0.3251
 is dichularly fiom. The dichald preanari. Arpand of Wichusbe, which make up 98.5% of Japan's citity


Iter-50000 loss: 0.3095
sidered implutareges to ed buntenjopula. Japanese country was reatariseviflital GDfean "S, the eamy 


Iter-51000 loss: 0.3050
e early werlare wich is the Upper Palec of rucleancy, utal come powhicOcyked Arially werotes finst t


Iter-52000 loss: 0.3086
ty of Japan's Touranas and an Chinase Werol ho worle's a of the First Sint-wrnten to the Sea, follec


Iter-53000 loss: 0.2874
an was inst9en a diveness Ripinm"rnase rund a lsered fowes peithe ward malitatinud dariss of the 12t


Iter-54000 loss: 0.2907
my by ploped in 1947, Japan has maimod for the Weijitativen red in the Country Brand SinoEas. Japan 


Iter-55000 loss: 0.2690
we derided intred int-ry hss es. Ancato-she on the core city of Tokyo, the carilatas eatieane with t


Iter-56000 loss: 0.2869
e forld hhige mokulitid the world's naseven hester Index. Japan'st ofochan surgast rigr

Iter-73000 loss: 0.2127
arji that 37 the Globan "end of istonter of 1945 wonhith to thar Tokokkedod offlica and so Empiticin


Iter-74000 loss: 0.2236
s of Hiroshima and Nagasaki. Since adoptingis the with the MentoliCaterea ie 19me Wing rechusefed Un


Iter-75000 loss: 0.2194
to and Wate o fle four landed oster is the rowes ors const te perdgr uste first the oster Pale dud c


Iter-76000 loss: 0.2297
 The first country in the world, as Runsicitoky po ullar ortec and fourth ly fourth-largest afoc an,


Iter-77000 loss: 0.2040
ed the Risstenflith the Summer a dath-sakyopod and Russo-Japanese War and World. The Seainimymbionol


Iter-78000 loss: 0.2096
an Developm bothe Second bury. Japan ento mernary with the world's eighth lead the world's fourth la


Iter-79000 loss: 0.1956
ond Sino-Japanese War and World War I allowed Japan to expand its empire a which was, is the nate 20


Iter-80000 loss: 0.2044
he 1st century, Aborth of the norg colutica ingt make up Japan's nameluring and Taiwan 

Iter-97000 loss: 0.1660
ke up 98.5% of Japan's total population. Ind the Paled it inchigh surced fionhiwad insu and Russity 


Iter-98000 loss: 0.1824
he world, as and dhigh surroliong the Sea of Okhotsk in the nornd lowed purced. The first in the num


Iter-99000 loss: 0.1620
 of the UN, the G7, the G8, and the Emperor as a der. Japan is ragked first in 1868 the ailitaky fow


Iter-100000 loss: 0.1632
the early 17th century, which wastol thich istred's livodeat China, followed by periods. The popban 


Iter-101000 loss: 0.1764
ple livedel fouht 2015–2016 and is the highest life OEped of the fourth ly remion egropurregipings a


Iter-102000 loss: 0.1354
h the fits laran searunity. Alome tury, which came the 126 and early buntirsto with in the land coun


Iter-103000 loss: 0.1810
r Iely bovel Chendere feutataineagas uropudx, vint ovestth to the Ease probal milital restored is a 


Iter-104000 loss: 0.1836
leet pressixiveres of incendst ries, thes, malle toecteity in the country is divid

...

Iter-889000 loss: 0.0806
d country in Asia. Japan is ranked first in the Country Brand Index, ranked sixth in the Global Comp


Iter-890000 loss: 0.0523
urth-largest economy by purchasing power parity. It is also the world's eighth largest military budg


Iter-891000 loss: 0.1139
te in 1853 when a United States fleet pressured Japan was rustored as Tonal Diet. Japan is a member 


Iter-892000 loss: 0.0889
g prefectures, is the world's largest metropolitan area with over 35 million residents and the world


Iter-893000 loss: 0.0727
arly 17th century, which was ended in 1853 when a United Olympo d c ther himborth largest GDP and th


Iter-894000 loss: 0.1248
officially renounced its right to the east of the Russo-Japanese War of 1937 expandexUwes the and fo


Iter-895000 loss: 0.0900
 a divine periods of isolation in the early 17th century CE. Influence from of revel ushuned first i


Iter-896000 loss: 0.0718
illion residents and an elected legislature called the Nath the Embings of inte

Iter-913000 loss: 0.0865
Honshu, Hokkaido, Kyushu and Shikoku, which make up aboun, is the world's largest metropolitan area 


Iter-914000 loss: 0.0601
 East As.boTh orictame to wirea, which includes Tokyo and tohe the Meiji Emperor was restored as hea


Iter-915000 loss: 0.0829
ng the has maintained a unitary constitutional mainly Imperial Chich in the name of the Emperor. Jap


Iter-916000 loss: 0.1118
icularly from Wintory, which make up 98.5% of Japan's total population. Influencer with uized Hudxca


Iter-917000 loss: 0.0755
 percent of Ji. Nepido, Kyushu and Shikoku, which make up about ninety-seven percent of Japan's land


Iter-918000 loss: 0.0510
ountry is divided into 47 prefectures, is the world's lare Honsh make up the 1st century CE. Inflic 


Iter-919000 loss: 0.1177
evised constitutiot TolacSectand peigity in the are Hinomand it is often called the "Land of the Rep


Iter-920000 loss: 0.1000
the 12th century until 1868, Japan was the first country in Asia with over 35 m

Iter-937000 loss: 0.1128
ECD and the fourth leading global conflict and inse hich make up 98.5% of Japan's total population. 


Iter-938000 loss: 0.0923
-koku or Nihon-koku, "loby Japanese: 日本 Nippon or Nihon; formally. About this sound Nippon-koku or N


Iter-939000 loss: 0.0723
purchasing power parity. It is also the world's largest urban agglomeration economy. Archaeological 


Iter-940000 loss: 0.1267
fore the Meiji Emperor was restored as head of state in 1868 andesterized Japan's history. From the 


Iter-941000 loss: 0.0922
 Area, which includes Tokyo and several surrounding prefectures in eight regions, maintained a unita


Iter-942000 loss: 0.0830
ia. Located in the Pacific Oceatel unficiallowed Japan's history. From the 12th century untic popula


Iter-943000 loss: 0.1081
r Tokyo and several surrounding prefecturing frected of the UN, thees of Japan's land area. The coun


Iter-944000 loss: 0.0977
mpire of Japan was proclaimed, with the Emperor as has of lintovoft 201lan laue

Iter-961000 loss: 0.1054
xts from the 1st century CE. Influence from other regions, mainly Imperial China, Korea and SPalepin


Iter-962000 loss: 0.0973
rchipelagon 6ulity. It ingst by nouthiman icly 9. Japan entered a gan adco calmereainliaid its empir


Iter-963000 loss: 0.0940
e expectancy, the third lowest infant mortality in 1947, Japan has maintained a unitary constitution


Iter-964000 loss: 0.0895
r II in 1941, which came to an end in 1945 followins a morth latior militar2 dulle by of cons the hi


Iter-965000 loss: 0.1044
red period. The first written, with the 1st century, which was end of sedgst both the Summer lag or 


Iter-966000 loss: 0.0801
ushu and Shikoku, which make up 98.5% of Japan's total pometr loun in the Global Competitiveness Rep


Iter-967000 loss: 0.0950
ed first in the Country Brand Index, Russ in the south. The kanji that make up Japan, peaclion orten


Iter-968000 loss: 0.0944
gs of Hiros, was in the ealoporten Dy bud eas followed by China Sea and Taiwan 

Iter-985000 loss: 0.1014
d States fleet pressured Japan was rinsted from the 1st century CE. Influence from other regions, ma


Iter-986000 loss: 0.0853
n was the first country in East Asia. Located in the Pacific Ocean, it lies to the east of the Sea o


Iter-987000 loss: 0.1037
Nippon or Nihon; forly twhine from other regions. The population enjoys the Risilapie lire dollound 


Iter-988000 loss: 0.0958
nal GDP and the world's fourth-largest exporter and fourth-largest exporter and fourth-largest expor


Iter-989000 loss: 0.1033
 the atest cenf-lSeconomided in 1853 when a United and Russia, stretching from the Summer leas the w


Iter-990000 loss: 0.0930
eading global city in the Global Competites frized Japan was ruled in the number of Nobel laun Emper


Iter-991000 loss: 0.0934
 "State of. o and eapl and World War II in 1941, which came to an is a stral Comped country in Asia 


Iter-992000 loss: 0.1002
ity. It is also the world's largest metroped in the Pacific Ocean, it lies to t

In [ ]:
=========================================================================
Iter-999000 loss: 0.0902
=========================================================================
Sea, China, Korea and Russia, stretching from the Sea of Okhotsk in the Global impopticuch censure w
=========================================================================


=========================================================================
Iter-1000000 loss: 0.1001
=========================================================================
e war, it maintains a modern mic the world's fourth-largest economy by purchalisurre country is divi
=========================================================================


Out[8]:
<__main__.GRU at 0x10d1c2fd0>

In [ ]:
=========================================================================
Iter-148000 loss: 0.1209
=========================================================================
land 1868, Japan is a st in CEm Niprover was the Upprocoun, the nasiof limperomonsof-useates of the 
=========================================================================


=========================================================================
Iter-149000 loss: 0.1198
=========================================================================
 a is the highest milliont an suust. The Grory. Abothan wres hesth largest are Honshu, Hokeke, shufr
=========================================================================


=========================================================================
Iter-150000 loss: 0.1197
=========================================================================
's history. From Ware war, st ricity of Japan") is an islands in the Ochas of sixtht Sea of 1shict u
=========================================================================


Out[21]:
<__main__.GRU at 0x10d0830f0>

In [ ]:
=========================================================================
Iter-99000 loss: 0.1560
=========================================================================
 of the UN, the G7, the G8, and the world's fourth-largest economy, wites ecatroped bote period. Tov
=========================================================================


=========================================================================
Iter-100000 loss: 0.1881
=========================================================================
the worth-leven periotan and in 1945 following the atom inter and Resilled the "Land of the Rssicand
=========================================================================


Out[13]:
<__main__.GRU at 0x10d084470>

In [ ]:
=========================================================================
Iter-297000 loss: 0.1208
=========================================================================
ry with a Unised Japan's history. From the 12th century, which isHomAreat Divi al China, folicter Ol
=========================================================================


=========================================================================
Iter-298000 loss: 0.0927
=========================================================================
 the number of Nobel laureates of anI arean Index. Japan was the first cores. Japan is a developed c
=========================================================================


=========================================================================
Iter-299000 loss: 0.1143
=========================================================================
ake up Jcaly Id in 1868 and the Empire of Japan was proclaimed, ater popJdallky, thicalic and eanly 
=========================================================================


=========================================================================
Iter-300000 loss: 0.1084
=========================================================================
uled by suchaed into a and in 1945 follodud of Hurigh in the world. The Greater Tokyo Area, which in
=========================================================================


Out[27]:
<__main__.GRU at 0x10d06ed68>

In [ ]:

=========================================================================
Iter-198000 loss: 0.1331
=========================================================================
tary shoguns who ruled in the nainty n mainterd Taiwat lors latulaty und Sunarend thia dudg oonaly r
=========================================================================


=========================================================================
Iter-199000 loss: 0.1386
=========================================================================
th largest imf in the korlation mainlatI un the eipan endand of the Rising Sun". Japan is a stratovo
=========================================================================


=========================================================================
Iter-200000 loss: 0.0987
=========================================================================
hest-ranked Asian countrord with the Since caly towertor Jaty in the country in East Asia. Located i
=========================================================================


Out[11]:
<__main__.GRU at 0x10d053f98>

In [ ]:
=========================================================================
Iter-99000 loss: 0.1890
=========================================================================
 of No20th bex, mfen eped consic th led the fourth leading globan in the name of the Emperor. Japan 
=========================================================================


=========================================================================
Iter-100000 loss: 0.1673
=========================================================================
the early 1shthicaped and the forchest urerlate 19th and the world's fourth-largest economy by purch
=========================================================================


Out[24]:
<__main__.GRU at 0x10d1f7b00>

In [ ]:
=========================================================================
Iter-100000 loss: 0.3913
=========================================================================
the 1pman a inusy rotory surrowho tOkike upporter wor peven des The easty porterized itivecend perce
=========================================================================


Out[7]:
<hipsternet.neuralnet.RNN at 0x7fc2f5182e80>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1615
=========================================================================
the name of the 12th century until 1868, Japan was ruled by successime Wer of 1941, which country is
=========================================================================


Out[16]:
<hipsternet.neuralnet.LSTM at 0x7f67fe0c9978>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1333
=========================================================================
the Coun resilate a in the population. Russided in the OECD and the world's fourth-largest econompic
=========================================================================


Out[10]:
<hipsternet.neuralnet.GRU at 0x7fc2cec1e1d0>